In [ ]:
%env OPENAI_API_KEY=keyhere

In [17]:
from datetime import datetime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

from kagent.tools import BuiltInTool

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message=f"""
    You are a planning agent. Current date and time is: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
    Your job is to break down complex tasks into smaller, manageable subtasks that can be executed by the team members. DO NOT MAKE UP ADDITIONAL AND UNNECESSARY SUBTASKS.
    Your team members are:
        prometheus_agent: Knows how to query and retrieve metrics for services in the cluster.
        istio_agent: Knows how to work with Istio CLI and manage Istio installations.
        k8s_agent: KNows how to interact with a Kubernetes cluster using the kubectl CLI. It can only run kubectl commands.

    You only plan and delegate tasks - you do not execute them yourself. Always assign the task to the most appropriate team member.
    Each team member has a specific set of skills and tools they can use to complete the task and provide the necessary information to you.
    Tasks that explicitly require a specific tool or agent should be assigned to the agent with that tool. For example:
    - if the task requires querying for metrics, it should be assigned to the Prometheus agent.
    - if the task requires to install Istio, it should be assigned to the Istio agent.

    When assigning tasks, use this format:
    1. <agent>: <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

istioctl_agent = AssistantAgent(
    "istioctl_agent",
    description="An agent for Istio operations",
    tools=[BuiltInTool("istio.install"), BuiltInTool("istio.verify_install")],
    model_client=model_client,
    system_message=f"""
    You are an Istio agent that knows how to install and verify Istio installation.
    Current date and time is: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
    """,
)

prom_agent = AssistantAgent(
    "prometheus_agent",
    description="An agent for Prometheus operations",
    tools=[
        BuiltInTool("prometheus.query"),
        BuiltInTool("prometheus.query_range"),
        BuiltInTool("prometheus.get_series"),
        BuiltInTool("prometheus.get_label_names"),
        BuiltInTool("prometheus.get_label_values"),
    ],
    model_client=model_client,
    system_message=f"""
    You are a Prometheus agent. You know how to query metrics from Prometheus. Current date and time is: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

    Before executing any tools, check whether Istio is installed and based on that use the appropriate metric name (e.g. `istio_requests_total`).
    """,
)


k8s_agent = AssistantAgent(
    "k8s_agent",
    description="An agent for k8s operations",
    tools=[BuiltInTool("k8s.get_resources")],
    model_client=model_client,
    system_message="""
    You are a k8s agent. You know how to interact with a Kubernetes cluster.
    """,
)


# istio_agent = AssistantAgent(
#     "istio_agent",
#     description="An agent for creating Istio resources",
#     tools=[BuiltInTool("istio.generate_resource")]
#     model_client=model_client,
#     system_message="""
#     You are an Istio agent that knows how to generate YAML for Istio.
#     You only respond with YAML for Istio resources and nothing else. If you don't know the answer, say "I don't know".

#     The resources you can generate are:
#     - Gateway
#     - VirtualService
#     - AuthorizationPolicy
#     - PeerAuthentication
#     """,
# )

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


team = SelectorGroupChat(
    [planning_agent, k8s_agent, prom_agent],
    model_client=model_client,
    termination_condition=termination,
    allow_repeated_speaker=True,
)

task = "I have a bunch of pods running in default namespace. Can you show me the metrics and create a dashboard?"  # , then install Bookinfo (https://raw.githubusercontent.com/istio/istio/refs/heads/master/samples/bookinfo/platform/kube/bookinfo.yaml)"

# Use asyncio.run(...) if you are running this in a script.
# await Console(team.run_stream(task=task))


---------- user ----------
I have a bunch of pods running in default namespace. Can you show me the metrics and create a dashboard?
---------- PlanningAgent ----------
To fulfill your request, we need to complete the following subtasks:

1. Retrieve the list of pods running in the default namespace.
2. Query metrics for those pods.
3. Create a dashboard to display the metrics.

Here is how we will divide the tasks:

1. k8s_agent: Retrieve the list of pods currently running in the default namespace.
2. prometheus_agent: Query and retrieve metrics for the pods running in the default namespace.
3. prometheus_agent: Create a dashboard using the retrieved metrics.

Let's proceed with these tasks.
---------- k8s_agent ----------
[FunctionCall(id='call_VSg1zsHoWN9yLluB2QhHRqCQ', arguments='{"name":"pods","resource_type":"pods","ns":"default","output":"wide"}', name='get_resources')]
---------- k8s_agent ----------
[FunctionExecutionResult(content='Error running kubectl command: Error from ser

/Users/peterj/projects/kagent/python/.venv/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:416: UserWarning: Both tool_calls and content are present in the message. This is unexpected. content will be ignored, tool_calls will be used.
  model_result = await self._model_client.create(


---------- k8s_agent ----------
[FunctionCall(id='call_Ox4XuPwH2eFCKSi8iyzjja4e', arguments='{"name":"","resource_type":"pods","ns":"default","output":"wide"}', name='get_resources')]
---------- k8s_agent ----------
[FunctionExecutionResult(content='NAME                                     READY   STATUS    RESTARTS        AGE     IP            NODE                 NOMINATED NODE   READINESS GATES\ncurl                                     1/1     Running   1 (2d14h ago)   2d15h   10.244.0.37   kind-control-plane   <none>           <none>\ndetails-v1-65cfcf56f9-x9vlv              1/1     Running   0               2d15h   10.244.0.32   kind-control-plane   <none>           <none>\nhttpbin-gateway-istio-5bfbd7f47f-lqbrh   1/1     Running   0               2d14h   10.244.0.40   kind-control-plane   <none>           <none>\nproductpage-v1-d5789fdfb-bw9dh           1/1     Running   0               2d15h   10.244.0.36   kind-control-plane   <none>           <none>\nratings-v1-7c9bd4b87f-v9mx

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x111939340>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x1120af2f0>, 3191299.096856833)])']
connector: <aiohttp.connector.TCPConnector object at 0x1119392b0>
/Users/peterj/projects/kagent/python/.venv/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:416: UserWarning: Both tool_calls and content are present in the message. This is unexpected. content will be ignored, tool_calls will be used.
  model_result = await self._model_client.create(
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1119cd670>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x1120af0b0>, 3191308.153203541)])']
connector: <aiohttp.connector.TCPConnector object at 0x1119cc7d0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1119ce600>
Unclosed connector
conne

CancelledError: 